## Dependencies

---

In [ ]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


In [ ]:
# edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
edge.maximize_window()

## Extract

---

### Functions:

In [ ]:
def resources(name: str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content: dict[str:str] = yaml.safe_load(f)
        return content[name]

In [ ]:
def close_prompt(driver: object) -> None:
    """Close ad window"""

    button: str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [ ]:
def choose_category(category: str, driver: object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [ ]:
def click_element(element: object, path: str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [ ]:
def extract_feature_in(element: object, path: str) -> str:
    """Extract ad feature where current browser page is in"""

    feature: str = element.find_element(by=By.XPATH, value=path)
    return feature

In [ ]:
def extract_feature_out(element: object, path: str) -> list[str]:
    """Extract ad feature from list of ads (not within the ad)"""

    feature = []

    try:
        feature: str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [ ]:
def persistence(dataframe: pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [ ]:
def get_len(features: list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
            lengths.append(len(feature))
    return lengths

In [ ]:
def check_len(lengths: list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

Variables pool.

In [ ]:
descriptions: list[str] = []
prices: list[int] = []
models: list[str] = []
years: list[int] = []
kilometers: list[int] = []
engines: list[str] = []

# seconds
DELAY: int = 1
SEDAN = resources("cars_sedan")

In [ ]:
SEDAN["year"]

Get web & close initial prompt (if is required).

In [ ]:
edge.get(SEDAN["url"])
# close_prompt(edge)

Extract car features.

In [ ]:
elements = edge.find_elements(by=By.XPATH, value=SEDAN["ad"])
elements[0].click()

try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(
            EC.presence_of_element_located((By.XPATH, SEDAN["description"])))

        brand_str = extract_feature_in(edge, SEDAN["brand"]).text
        model_str = extract_feature_in(edge, SEDAN["model"]).text
        models.append(brand_str + " " + model_str)

        descriptions.append(extract_feature_in(edge, SEDAN["description"]).text)
        prices.append(extract_feature_in(edge, SEDAN["price"]).text)
        kilometers.append(extract_feature_in(edge, SEDAN["kilometers"]).text)
        engines.append(extract_feature_in(edge, SEDAN["engine"]).text)
        years.append(extract_feature_in(edge, SEDAN["year"]).text)

        click_element(edge, SEDAN["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, SEDAN["exit"])

## Export

---

### Functions:

In [ ]:
def parse_price(dataframe: pd.DataFrame, feature: str, pattern: str) -> None:
    """
    Parse financial features like prices. Works with $#,### (not dot)
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe[feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",", "")))


In [ ]:
def remove_chars(dataframe: pd.DataFrame, feature: str, wildcards: str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        , axis=1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [ ]:
PRICE_RG: str = r"^\$(?P<prices>\d+,?\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG: str = '(\n|\r)+'

Check that all features has the same length.

In [ ]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices), len(descriptions)]
lengths

Create dataframe to transform data extracted

In [24]:
if check_len(lengths):
    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    persistence(df_temp)


In [25]:
df_temp

,model,year,kilometers,engine,price,description
0,Mitsubishi Lancer,2017,68000,Gasolina,8400,Color : Azul Vendo Mitsubishi Lancer 2017 Full...
1,Mitsubishi Mirage G4,2018,60000,Gasolina,8400,Color : Blanco Se vende Mitsubishi G4 2018. Au...
2,Nissan Altima,2005,112000,Gasolina,3800,Precio negociable: si Color : Rojo Vendo Bonit...
3,Nissan Sentra,2015,16100,Gasolina,7200,Precio negociable: si Color : Rojo DESCRIPCIÓN...
4,Hyundai Elantra,2015,98000,Gasolina,9100,Financiamiento: no Precio negociable: si Color...
5,Kia Rio,2019,107000,Gasolina,7699,Precio negociable: si Kia Rio 2019 en camino a...
6,Toyota Corolla,2007,128,Gasolina,5500,DESCRIPCIÓN CALIFICACIONES SEGURIDAD Se vende ...
7,Kia Forte,2018,69038,Gasolina,8950,Color : negro Vendo kia forte 2018 motor 2.0 c...
8,Hyundai Accent,2007,155000,Gasolina,4600,Precio negociable: si Color : Gris rata DESCRI...
9,Hyundai Elantra,2015,150365,Gasolina,7975,Color : ROJO DESCRIPCIÓN CALIFICACIONES SEGURI...
